In [ ]:
from asim.dataset.scene.scene_builder import ArrowSceneBuilder
from asim.dataset.scene.scene_filter import SceneFilter

from nuplan.planning.utils.multithreading.worker_sequential import Sequential
# from nuplan.planning.utils.multithreading.worker_ray import RayDistributed

In [ ]:
split = "nuplan_mini_val"

log_names = None
scene_tokens = None

# log_names = ["2021.06.07.12.54.00_veh-35_01843_02314"]
# scene_tokens = ["2283aea39bc1505e"]

scene_filter = SceneFilter(
    split_names=[split],
    log_names=log_names,
    scene_tokens=scene_tokens,
    duration_s=15.1,
    history_s=1.0,
)
scene_builder = ArrowSceneBuilder("/home/daniel/asim_workspace/data")
worker = Sequential()
# worker = RayDistributed()
scenes = scene_builder.get_scenes(scene_filter, worker)

print(f"Number of scenes: {len(scenes)}")


In [ ]:
from asim.simulation.controller.action_controller import ActionController
from asim.simulation.controller.motion_model.kinematic_bicycle_model import KinematicBicycleModel
from asim.simulation.observation.log_replay_observation import LogReplayObservation
from asim.simulation.simulation_2d_setup import Simulation2DSetup
from asim.simulation.simulation_2d import Simulation2D


from asim.simulation.simulation_time_controller.step_simulation_time_controller import StepSimulationTimeController


def get_simulation_2d_setup(scene):
    return Simulation2DSetup(
        time_controller=StepSimulationTimeController(scene),
        observations=LogReplayObservation(),
        ego_controller=ActionController(scene, KinematicBicycleModel()),
        scene=scene,
    )


idx = 1337
simulation_2d_setup = get_simulation_2d_setup(scenes[idx])
simulation2d = Simulation2D(simulation_2d_setup)

In [ ]:
# reset
from asim.simulation.planning.planner_output.action_planner_output import ActionPlannerOutput


planner_initialization = simulation2d.initialize()
planner_input = simulation2d.get_planner_input()


# TODO: Implement action planner output
# TODO: Further test the simulation object.

action = ActionPlannerOutput()

#
for i in range(10):
    # step
    simulation2d.propagate()

    # get planner input
    planner_input = simulation2d.get_planner_input()

    # get action
    action = simulation2d.get_action(planner_input)

    # apply action
    simulation2d.apply_action(action)